## Include common.ipynb

In [1]:
from collections import OrderedDict
from typing import List, Tuple

import numpy as np
import torch

from flwr.common import Context, Metrics
from flwr.server import start_server, ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg

## Flower Server

In [2]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [3]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    )

    config = ServerConfig(num_rounds=1)

    return strategy, config


strategy, config = server_fn(context=None)

start_server(
    server_address="127.0.0.1:8081",
    config=config,
    strategy=strategy
)

INFO :      Starting Flower server, config: num_rounds=1, no round_timeout
INFO :      Flower ECE: gRPC server running (1 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 1 round(s) in 22.93s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.05366434156894684
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.39909999999999995)]}
INFO :      


History (loss, distributed):
	round 1: 0.05366434156894684
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.39909999999999995)]}